In [341]:
from google.cloud import firestore
import google.cloud.exceptions

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/277594b/Documents/Projects/COKI/certs/coki-214004-36a73c3216c6.json"

In [342]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [343]:
db = firestore.Client()

In [445]:
task_error_docs = db.collection("tasks").where(u'state', u'==', "error").get()

In [446]:
breakdown = {
    "msa": {
        "fetch": [],
        "parse": [],
        "load": []
    },
    "wos": {
        "fetch": [],
        "parse": [],
        "load": []
    },
    "scopus": {
        "fetch": [],
        "parse": [],
        "load": []
    }
}

for task in task_error_docs:
    task_dict = task.to_dict()
    if task_dict["fetch"]["blob_name"] != 'None':
        if task_dict["parse"]["blob_name"] != 'None':
            if "results" in task_dict["load"]:
                if task_dict["load"]["results"] > 0:
                    print(task.id, "happy")
                else:
                    breakdown[task_dict["source"]]["load"].append(task_dict)
            else:
                breakdown[task_dict["source"]]["load"].append(task_dict)
        else:
            breakdown[task_dict["source"]]["parse"].append(task_dict)
    else:
        breakdown[task_dict["source"]]["fetch"].append(task_dict)

In [447]:
for key in breakdown:
    print(key)
    for other_key in breakdown[key]:
        print(other_key + " error:", len(breakdown[key][other_key]))
    print("")

msa
fetch error: 0
parse error: 0
load error: 0

wos
fetch error: 0
parse error: 0
load error: 0

scopus
fetch error: 0
parse error: 0
load error: 0



In [49]:
for key in breakdown:
    for error in breakdown[key]["load"]:
        print(error)
        print("")
        print(error["parse"]["blob_name"])
        print(error["load"]["bigquery_job_id"] if "bigquery_job_id" in error["load"] else "none")
        print("")

{'load': {'timestamp': '2018-11-21 05:36:20.048306+00:00', 'bigquery_job_id': '277564dd-61d7-4eac-8ae8-f3565cb4b43f', 'eventId': ''}, 'gridId': 'grid.83440.3b', 'previousTask': None, 'active': True, 'fetch': {'timestamp': '2018-11-21 05:36:20.048306+00:00', 'eventId': '', 'blob_name': 'v1/grid.83440.3b/msa/2014-01/2018-11-21 05:36:20.048306+00:00_a09acf89-4ffa-472e-8ab7-02f0d37742e0.json'}, 'source': 'msa', 'state': 'error', 'dispatch': {'timestamp': '2018-11-21 05:36:19.990416+00:00', 'eventId': 'scheduler_placeholder_id', 'pubsub_message_id': '302616794010999'}, 'tasking_id': 'c862c7b3-593e-46f5-82da-2e47b9a0da88', 'parse': {'eventId': 'placeholder', 'blob_name': 'v1/grid.83440.3b/msa/2014-01/2018-11-21 05:36:20.048306+00:00_a09acf89-4ffa-472e-8ab7-02f0d37742e0.csv', 'timestamp': '2018-11-21 05:36:20.048306+00:00'}, 'created': '2018-11-21 04:01:38.712607+00:00', 'externalIdentifier': '45129253', 'month': '2014-01'}

v1/grid.83440.3b/msa/2014-01/2018-11-21 05:36:20.048306+00:00_a09acf

In [310]:
single = db.collection("tasks").document("00503459-f7be-4ff3-8d84-e6276905f18a").get()

In [311]:
single_dict = single.to_dict()
pp.pprint(single_dict)

{   'active': True,
    'created': '2018-11-28 06:09:59.171302+00:00',
    'dispatch': {   'errorMessage': '',
                    'eventId': 'scheduler_placeholder_id',
                    'pubsub_message_id': '308833272721452',
                    'state': 'complete',
                    'timestamp': '2018-11-28 15:19:05.873670+00:00'},
    'externalIdentifier': '60007457',
    'fetch': {   'blob_name': 'v1/grid.14476.30/scopus/2011-10/2018-11-28 '
                              '15:19:05.898722+00:00_00503459-f7be-4ff3-8d84-e6276905f18a.json',
                 'errorMessage': '',
                 'eventId': '',
                 'state': 'completed',
                 'timestamp': '2018-11-28 15:19:05.898722+00:00'},
    'gridId': 'grid.14476.30',
    'lastUpdated': '2018-11-30 03:55:39.023592+00:00',
    'load': {   'bigquery_job_id': '55b16975-32a8-4d7d-a2b4-a267a90c4d3b',
                'errorMessage': '',
                'eventId': '',
                'results': 320,
             

In [128]:
sp_task_error_docs = db.collection("tasks").where(u'state', u'==', "error").where(u'gridId', u'==', "grid.4305.2").get()

In [129]:
sp_errors = {}
for task in sp_task_error_docs:
    task_dict = task.to_dict()
    sp_errors[task.id] = task_dict

In [130]:
len(sp_errors)

1

In [131]:
for key, error in sp_errors.items():
    if len(error["fetch"]["blob_name"]) < 10:
        print(key)
        #db.collection("tasks").document(key).update({
        #    u"state": u"open"
        #})

a55e7f4e-d171-4e6c-8930-3556dc2510c9


In [106]:
sp_task_error_docs = db.collection("tasks").where(u'state', u'==', "error").where(u'source', u'==', "wos").where(u'gridId', u'==', "grid.116068.8").get()

sp_errors = {}
for task in sp_task_error_docs:
    task_dict = task.to_dict()
    sp_errors[task.id] = task_dict

print(len(sp_errors))
for key, error in sp_errors.items():
    if len(error["fetch"]["blob_name"]) < 10:
        print(key)
        db.collection("tasks").document(key).update({
            u"state": u"open",
            u'externalIdentifier': 'Massachusetts Institute of Technology MIT'
        })

60
11e886bb-e67c-4d3a-95eb-dcbe76810ec2
16a382bf-df2b-4252-adcb-3feb0ba12866
1bf53981-9701-48a8-a931-40c3d7669ee4
1dad0943-cc2b-44e8-b4e9-67002673a242
203970c2-309e-4393-a482-9727a79741ba
21d12ed8-6c13-4fbd-aa4d-cdad7182168b
2419bdd8-e734-47fe-8c1b-cfb78a54b3da
25181700-db07-494e-98dd-060fa8ca743b
262f5dd3-6f37-4044-bc8c-0627a7b0f7bd
2a5fd50b-8290-4039-8fb2-d2203a8281f2
36b347bc-b6fc-4aa5-a9a6-477f2458c8e3
37e86913-3da2-4f09-b80c-f3cbb44de6af
3a16014b-97c6-434d-8dfe-4e9fa0ca0e5d
3ac3cb6f-3838-475e-b845-a28b5b0ead41
4de675b7-4e9c-48ea-840c-ba7bd76dbd98
5070fa48-cb71-4f58-a5ae-6e13fcad4d98
53d3c27c-4618-4081-99da-3578508b33ea
54b3d385-f769-406e-9d2a-7f2e3d005e1e
5703cfe8-c358-430b-a10a-3e3b42e6dcc8
6603bce6-94df-4fc7-bedb-dd83069431e4
69a5fb4c-9d63-4b4e-bb8e-76588700219f
6b5912ac-4564-460f-848f-305d257d6d92
6bcdbab8-3de3-45b4-840a-1f47ee35c736
6cdc61e2-0c57-4d0c-980c-2c3452444493
6d2edf8b-d551-4a16-98a6-8258c87a4061
7496e853-e953-471a-8f96-3bcedd403b4a
77fb7548-be07-4b67-aba8-d020ddb09a1

In [103]:
pp.pprint(sp_errors["11e886bb-e67c-4d3a-95eb-dcbe76810ec2"])

{   'active': True,
    'created': '2018-11-21 04:01:33.082225+00:00',
    'dispatch': {   'eventId': 'scheduler_placeholder_id',
                    'pubsub_message_id': '302757443751688',
                    'timestamp': '2018-11-21 10:27:14.897849+00:00'},
    'externalIdentifier': 'Massachusetts Institute of Technology (MIT)',
    'fetch': {   'blob_name': 'None',
                 'eventId': '',
                 'timestamp': '2018-11-21 10:27:14.908978+00:00'},
    'gridId': 'grid.116068.8',
    'load': {   'bigquery_job_id': '',
                'function_id': '',
                'results': 0,
                'timestamp': ''},
    'month': '2014-03',
    'parse': {'blob_name': '', 'function_id': '', 'timestamp': ''},
    'previousTask': None,
    'source': 'wos',
    'state': 'error',
    'tasking_id': 'c862c7b3-593e-46f5-82da-2e47b9a0da88'}


In [107]:
sp_task_error_docs = db.collection("tasks").where(u'state', u'==', "error").where(u'source', u'==', "wos").where(u'gridId', u'==', "grid.34980.36").get()

sp_errors = {}
for task in sp_task_error_docs:
    task_dict = task.to_dict()
    sp_errors[task.id] = task_dict

print(len(sp_errors))
for key, error in sp_errors.items():
    if len(error["fetch"]["blob_name"]) < 10:
        print(key)
        db.collection("tasks").document(key).update({
            u"state": u"open",
            u'externalIdentifier': 'Indian Institute of Science IISC - Bangalore'
        })

60
0600b732-0d5f-4132-9c61-9d2083f6e0de
099d9dbe-00af-4f4d-b269-a0c547a3ebe8
0a641920-1639-4f23-b7d8-1b123d6a1c7e
0e87eed6-56a9-4d2e-9aa1-060d7166cfa9
0f9b5933-3caa-4aaa-b17d-bb5b9638cdb4
11de5d98-b006-477d-8246-dc365d88f141
13f84c55-f684-4d11-9ca3-37fa9b43f960
1712eceb-b1c4-49ed-a7b5-7708947e950c
1857de64-bc50-474a-b312-ce856c65576e
1e6356fe-96fd-42a1-b656-a94ef9d72d18
22e2431a-f8a4-4854-8e9e-92288082864b
24240f6e-0859-40ec-9513-6bdb47974214
32aa8cdd-b65d-4d29-be1c-eeea4d2f73ee
37196f96-526a-4473-930e-597deae7e373
3a289a23-97cd-419c-b0f5-d909cb8f4d36
44e92dae-7f13-48ad-a22d-da7b2890e8d2
5287c02e-227a-4cda-a9bb-610777145a4d
5342db94-7838-4e4b-b8da-74967f8b5b12
5914de83-284d-4c74-944c-ada22e3c85b4
59bb0c8e-d290-4ef3-91e3-310b543deb46
5e8cb10b-6cd6-4866-abaa-5c95e03b2371
678ff1b0-f4fd-4c8c-ab27-8f8134ffbb4b
6da36f97-2879-491e-b0c3-833a5aa3f6e8
725e49b3-b766-4dae-aa48-34c8b87bee55
74197054-ac0a-4290-8535-8de04c532bf1
77c555db-a991-4fb5-8aba-ffb2b500c088
7ab0bcf8-c325-4467-8560-dc829b7f17c

In [ ]:
15254493-f7a9-402c-ad0b-5d58063f7c9d

In [98]:
mit_task_error_docs = db.collection("tasks").where(u'gridId', u'==', "grid.116068.8").get()
mit = {}
for task in mit_task_error_docs:
    task_dict = task.to_dict()
    mit[task.id] = task_dict

In [99]:
len(mit)

125

In [134]:
db.collection("tasks").document("a55e7f4e-d171-4e6c-8930-3556dc2510c9").update({
    u"state": u"open"
})

update_time {
  seconds: 1543302650
  nanos: 420312000
}

In [216]:
wos_task_error_docs = db.collection("tasks").where(u'state', u'==', "error").where(u'source', u'==', "msa").get()

objects = []
wos_errors = {}
for task in wos_task_error_docs:
    task_dict = task.to_dict()
    #objects.append(task_dict["fetch"]["blob_name"])
    #print("")
    #wos_errors[task.id] = task_dict
    print(task_dict)
    print("")
    
#for x in sorted(objects):
#    print(x)

{'previousTask': None, 'active': True, 'fetch': {'timestamp': '2018-11-21 05:25:21.264615+00:00', 'eventId': '', 'blob_name': 'v1/grid.14476.30/msa/2015-01/2018-11-21 05:25:21.264615+00:00_9454c8af-ba49-4a1c-875d-3460730618e3.json'}, 'source': 'msa', 'state': 'error', 'dispatch': {'pubsub_message_id': '302605284672082', 'timestamp': '2018-11-21 05:25:20.635177+00:00', 'eventId': 'scheduler_placeholder_id'}, 'tasking_id': 'c862c7b3-593e-46f5-82da-2e47b9a0da88', 'parse': {'eventId': 'placeholder', 'blob_name': 'None', 'timestamp': '2018-11-21 05:25:21.264615+00:00'}, 'created': '2018-11-21 04:01:35.980292+00:00', 'externalIdentifier': '19880235', 'month': '2015-01', 'load': {'timestamp': '', 'results': 0, 'bigquery_job_id': '', 'function_id': ''}, 'gridId': 'grid.14476.30'}

{'source': 'msa', 'lastUpdated': '2018-11-27 05:53:51.238150+00:00', 'state': 'error', 'dispatch': {'eventId': 'scheduler_placeholder_id', 'pubsub_message_id': '302616794010999', 'timestamp': '2018-11-21 05:36:19.990

In [173]:
len(wos_errors)

1

In [174]:
fetch_fails = {}
for key, error in wos_errors.items():
    if len(str(error["fetch"]["blob_name"])) < 10:
        fetch_fails[key] = error

In [175]:
len(fetch_fails)

0

In [176]:
for key, error in wos_errors.items():
    print(key)
    print(error)
    print("")
    #db.collection("tasks").document(key).update({
    #    u"state": u"open",
    #    u"lastUpdated": error["created"]
    #})

6c99f141-423e-4358-99bf-2c1c4a4a23b7
{'parse': {'blob_name': 'v1/grid.7836.a/scopus/2017-10/2018-11-21 11:58:22.254580+00:00_6c99f141-423e-4358-99bf-2c1c4a4a23b7.csv', 'eventId': 'placeholder', 'timestamp': '2018-11-21 11:58:22.254580+00:00'}, 'created': '2018-11-21 04:01:47.279082+00:00', 'externalIdentifier': '60000356', 'month': '2017-10', 'load': {'timestamp': '2018-11-21 11:58:22.254580+00:00', 'bigquery_job_id': '3689e3bd-8576-4687-9a28-7f52ca86f1a8', 'eventId': ''}, 'gridId': 'grid.7836.a', 'previousTask': None, 'active': True, 'fetch': {'timestamp': '2018-11-21 11:58:22.254580+00:00', 'eventId': '', 'blob_name': 'v1/grid.7836.a/scopus/2017-10/2018-11-21 11:58:22.254580+00:00_6c99f141-423e-4358-99bf-2c1c4a4a23b7.json'}, 'source': 'scopus', 'state': 'error', 'dispatch': {'pubsub_message_id': '302807612009901', 'timestamp': '2018-11-21 11:58:22.243647+00:00', 'eventId': 'scheduler_placeholder_id'}, 'tasking_id': 'c862c7b3-593e-46f5-82da-2e47b9a0da88'}



In [177]:
from google.cloud import bigquery
client = bigquery.Client()

In [200]:
job_id = "3689e3bd-8576-4687-9a28-7f52ca86f1a8"
location = "us"

job = client.get_job(job_id, location=location)  # API request

# Print selected job properties
print('Details for job {} running in {}:'.format(job_id, location))
print('\tType: {}\n\tState: {}\n\tCreated: {}\n\tRows: {}'.format(
    job.job_type, job.state, job.created, job.output_rows))

print(job.error_result, job.errors)

Details for job 3689e3bd-8576-4687-9a28-7f52ca86f1a8 running in us:
	Type: load
	State: DONE
	Created: 2018-11-21 11:59:45.148000+00:00
	Rows: 257
None None <bound method _AsyncJob.done of <google.cloud.bigquery.job.LoadJob object at 0x120bbed30>>


In [185]:
task_error_docs = db.collection("tasks").where(u'state', u'==', "error").get()
task_errors = {}
for task in task_error_docs:
    task_dict = task.to_dict()
    #objects.append(task_dict["fetch"]["blob_name"])
    #print("")
    task_errors[task.id] = task_dict
print(len(task_errors))

28


In [186]:
failed_bq = {}
for key, task in task_errors.items():
    if len(task['load']['bigquery_job_id']) > 5:
        failed_bq[key] = task
print(len(failed_bq))

27


In [286]:
good = {}
bad_accounted_for = {}
bad = {}
good_tasks = {}

for key, task in failed_bq.items():
    job_id = task['load']['bigquery_job_id']
    location = "us"
    job = client.get_job(job_id, location=location)
    print('Details for job {} running in {} with task_id {}:'.format(job_id, location, key))
    try:
        output_rows = job.output_rows
    except:
        output_rows = 0
        
    if job.error_result != None:  
        #print('\tType: {}\n\tState: {}\n\tError: {}\n\tCreated: {}\n\tRows: {}'.format(
        #    job.job_type, job.state, job.error_result, job.created, output_rows))
        if job_id in accounted_for:
            bad_accounted_for[key] = job
        else:
            bad[key] = job
    else:
        good[key] = job
        good_tasks[key] = task
        print('\tType: {}\n\tState: {}\n\tError: {}\n\tCreated: {}\n\tRows: {}'.format(
            job.job_type, job.state, job.error_result, job.created, output_rows))

Details for job f0c84549-8cae-450f-b8d0-5bbf879bee47 running in us with task_id 1034eb22-d580-4f5a-93d3-078c74ea5500:
	Type: load
	State: DONE
	Error: None
	Created: 2018-11-22 00:46:34.658000+00:00
	Rows: 446
Details for job 4127e277-b9c7-4d01-8aa1-2b5e810ea39f running in us with task_id 132068ad-90b4-423e-a3a8-917aaecbb571:
Details for job 08d11c4b-2da3-488a-9623-f051877555e5 running in us with task_id 1c1ac2b0-6af9-4776-b5cf-199f830d3f35:
Details for job 2c75f38a-2480-4a8f-95d1-9ead1c9f56d9 running in us with task_id 2b5225ae-4573-4031-bdcb-7aa6b7862b92:
Details for job f16ff213-f0ed-46e2-96d3-d89a30394cad running in us with task_id 32aa8cdd-b65d-4d29-be1c-eeea4d2f73ee:
	Type: load
	State: DONE
	Error: None
	Created: 2018-11-26 08:39:23.625000+00:00
	Rows: 230
Details for job 1d430d3b-2602-4e96-8a9e-357373246dfb running in us with task_id 3aab61cd-ddf2-4ab2-9cd2-090be11c35c3:
Details for job 56009cea-960d-49fc-b2cb-f5082e841b27 running in us with task_id 3d5487bf-1eb8-4108-9ce2-c8f2

In [209]:
#Details for job f20f92dc-fcba-4784-b619-35ded1f8bae9 running in us with task_id 553a6137-3724-41ff-b349-e7886f05b26d:
#Details for job 873eb4d6-1885-41bd-bff8-7b8e0bdab422 running in us with task_id a09acf89-4ffa-472e-8ab7-02f0d37742e0:
#Details for job 629ee755-ea27-4d9d-a559-86c5d4b93dfa running in us with task_id 9f24b372-7fa1-45c3-86e5-e8fc06072864:
#Details for job 1fc084d4-8d9b-4d8f-bf09-9aa584cf094f running in us with task_id f7f32a98-136b-4b03-98b7-5047f6dbb17f:
#Details for job 7b5e73c7-14eb-415f-81ae-9859dd080782 running in us with task_id 74d9a525-c167-4979-b6fa-5ba41b393ab4:
#Details for job 9a31b0b6-5afd-455d-9d39-5195f9b308e7 running in us with task_id b4d201f2-337f-4b87-946f-3b566b14bd87:
#Details for job 4a4916e0-6a39-4a10-9049-51a0c78bf887 running in us with task_id 48b62216-4a53-4454-ae49-fe34a7e04d45:

accounted_for = ["f20f92dc-fcba-4784-b619-35ded1f8bae9", "873eb4d6-1885-41bd-bff8-7b8e0bdab422", 
                 "629ee755-ea27-4d9d-a559-86c5d4b93dfa", "1fc084d4-8d9b-4d8f-bf09-9aa584cf094f",
                 "7b5e73c7-14eb-415f-81ae-9859dd080782", "9a31b0b6-5afd-455d-9d39-5195f9b308e7",
                 "4a4916e0-6a39-4a10-9049-51a0c78bf887"]

In [214]:
print(len(good))
print(len(bad_accounted_for))
print(len(bad))

4
7
16


In [218]:
import json
with open("/Users/277594b/Documents/repos/coki-firebase/bigquery_resource__logs__2018-11-27T11-16.json") as f:
    data = json.load(f)

In [246]:
uris = []
for error in data:
    print(error['protoPayload']['serviceData']['jobCompletedEvent']['job']['jobConfiguration']['load']['sourceUris'][0])
    print(error['protoPayload']['serviceData']['jobCompletedEvent']['job']['jobStatus'])
    print("")
    uris.append(error['protoPayload']['serviceData']['jobCompletedEvent']['job']['jobConfiguration']['load']['sourceUris'][0])

gs://coki-parsed-doi/v1/grid.4305.2/wos/2008-08/2018-11-27 15:16:06.894621+00:00_3d5487bf-1eb8-4108-9ce2-c8f214e72328.csv
{'state': 'DONE', 'error': {'code': 3, 'message': 'Error while reading data, error message: CSV table encountered too many errors, giving up. Rows: 272; errors: 1. Please look into the error stream for more details.'}, 'additionalErrors': [{'code': 3, 'message': 'Error while reading data, error message: CSV table encountered too many errors, giving up. Rows: 272; errors: 1. Please look into the error stream for more details.'}, {'code': 3, 'message': 'Error while reading data, error message: Too many values in row starting at position: 50862.'}]}

gs://coki-parsed-doi/v1/grid.4305.2/wos/2007-11/2018-11-27 15:07:06.161231+00:00_1c1ac2b0-6af9-4776-b5cf-199f830d3f35.csv
{'state': 'DONE', 'error': {'code': 3, 'message': 'Error while reading data, error message: CSV table encountered too many errors, giving up. Rows: 40; errors: 1. Please look into the error stream for m

In [222]:
print(data[0]['protoPayload']['status'])

{'code': 3, 'message': 'Error while reading data, error message: CSV table encountered too many errors, giving up. Rows: 272; errors: 1. Please look into the error stream for more details.'}


In [237]:
import csv

csv_data = []

with open("/Users/277594b/Documents/repos/coki-firebase/job_understand.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    
    for row in reader:
        csv_data.append(row)


In [238]:
len(csv_data)

381

In [240]:
for row in csv_data:
    print(row)

OrderedDict([('10.1111/j.1471-8847.2007.00217.x', '10.1111/j.1558-5646.2008.00434.x'), ('', ''), ('grid.4305.2', 'grid.4305.2'), ('wos', 'wos'), ('2008-08', '2008-08'), ('2018-11-27 15:16:06.894621+00:00', '2018-11-27 15:16:06.894621+00:00'), ('3d5487bf-1eb8-4108-9ce2-c8f214e72328', '3d5487bf-1eb8-4108-9ce2-c8f214e72328')])
OrderedDict([('10.1111/j.1471-8847.2007.00217.x', '10.1016/j.wear.2007.12.015'), ('', ''), ('grid.4305.2', 'grid.4305.2'), ('wos', 'wos'), ('2008-08', '2008-08'), ('2018-11-27 15:16:06.894621+00:00', '2018-11-27 15:16:06.894621+00:00'), ('3d5487bf-1eb8-4108-9ce2-c8f214e72328', '3d5487bf-1eb8-4108-9ce2-c8f214e72328')])
OrderedDict([('10.1111/j.1471-8847.2007.00217.x', ''), ('', ''), ('grid.4305.2', 'grid.4305.2'), ('wos', 'wos'), ('2008-08', '2008-08'), ('2018-11-27 15:16:06.894621+00:00', '2018-11-27 15:16:06.894621+00:00'), ('3d5487bf-1eb8-4108-9ce2-c8f214e72328', '3d5487bf-1eb8-4108-9ce2-c8f214e72328')])
OrderedDict([('10.1111/j.1471-8847.2007.00217.x', ''), ('', 

In [241]:
from google.cloud import storage

In [242]:
def rename_blob(bucket_name, blob_name, new_name):
    """Renames a blob."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    new_blob = bucket.rename_blob(blob, new_name)

    print('Blob {} has been renamed to {}'.format(
        blob.name, new_blob.name))

In [245]:
rename_blob("coki-tmp", "output.csv", "output1.csv")

Blob output.csv has been renamed to output1.csv


In [247]:
len(uris)

24

In [248]:
uris[0]

'gs://coki-parsed-doi/v1/grid.4305.2/wos/2008-08/2018-11-27 15:16:06.894621+00:00_3d5487bf-1eb8-4108-9ce2-c8f214e72328.csv'

In [257]:
uris[0][73]

'1'

In [262]:
rename_blob("coki-fetched-doi", "v1/grid.4305.2/wos/2008-08/2018-11-27 15:16:06.894621+00:00_3d5487bf-1eb8-4108-9ce2-c8f214e72328.json", 
            "v1/grid.4305.2/wos/2008-08/2018-11-27 15:16:06.894622+00:00_3d5487bf-1eb8-4108-9ce2-c8f214e72328.json")


NotFound: 404 POST https://www.googleapis.com/storage/v1/b/coki-fetched-doi/o/v1%2Fgrid.4305.2%2Fwos%2F2008-08%2F2018-11-27%2015%3A16%3A06.894621%2B00%3A00_3d5487bf-1eb8-4108-9ce2-c8f214e72328.json/copyTo/b/coki-fetched-doi/o/v1%2Fgrid.4305.2%2Fwos%2F2008-08%2F2018-11-27%2015%3A16%3A06.894622%2B00%3A00_3d5487bf-1eb8-4108-9ce2-c8f214e72328.json: No such object: coki-fetched-doi/v1/grid.4305.2/wos/2008-08/2018-11-27 15:16:06.894621+00:00_3d5487bf-1eb8-4108-9ce2-c8f214e72328.json

In [278]:
for key, item in good_tasks.items():
    print(key, item)
    print("")
    db.collection("tasks").document(key).update({
        u"state": u"error-not_actually-failed",
        u"lastUpdated": item["created"]
    })

1034eb22-d580-4f5a-93d3-078c74ea5500 {'created': '2018-11-21 04:01:54.683597+00:00', 'externalIdentifier': 'Universidad Nacional Autonoma de Mexico', 'month': '2016-06', 'load': {'eventId': '', 'timestamp': '2018-11-22 00:46:24.204666+00:00', 'bigquery_job_id': 'f0c84549-8cae-450f-b8d0-5bbf879bee47'}, 'gridId': 'grid.9486.3', 'active': True, 'previousTask': None, 'fetch': {'timestamp': '2018-11-22 00:46:24.204666+00:00', 'eventId': '', 'blob_name': 'v1/grid.9486.3/wos/2016-06/2018-11-22 00:46:24.204666+00:00_1034eb22-d580-4f5a-93d3-078c74ea5500.json'}, 'source': 'wos', 'state': 'error', 'dispatch': {'eventId': 'scheduler_placeholder_id', 'pubsub_message_id': '303321092368381', 'timestamp': '2018-11-22 00:46:24.192535+00:00'}, 'tasking_id': 'c862c7b3-593e-46f5-82da-2e47b9a0da88', 'parse': {'eventId': 'placeholder', 'blob_name': 'v1/grid.9486.3/wos/2016-06/2018-11-22 00:46:24.204666+00:00_1034eb22-d580-4f5a-93d3-078c74ea5500.csv', 'timestamp': '2018-11-22 00:46:24.204666+00:00'}}

32aa8c

In [285]:
task_error_docs = db.collection("tasks").where(u'state', u'==', "error-not_actually-failed").get()
task_errors = {}
for task in task_error_docs:
    db.collection("tasks").document(task.id).update({
        #u"state": u"open",
        #u"lastUpdated": item["created"]
        u"load.results": 
    })
    print(task.id)
    print(task.to_dict())
    print("")

1034eb22-d580-4f5a-93d3-078c74ea5500
{'created': '2018-11-21 04:01:54.683597+00:00', 'externalIdentifier': 'Universidad Nacional Autonoma de Mexico', 'month': '2016-06', 'load': {'timestamp': '2018-11-22 00:46:24.204666+00:00', 'bigquery_job_id': 'f0c84549-8cae-450f-b8d0-5bbf879bee47', 'eventId': ''}, 'gridId': 'grid.9486.3', 'previousTask': None, 'active': True, 'fetch': {'eventId': '', 'blob_name': 'v1/grid.9486.3/wos/2016-06/2018-11-22 00:46:24.204666+00:00_1034eb22-d580-4f5a-93d3-078c74ea5500.json', 'timestamp': '2018-11-22 00:46:24.204666+00:00'}, 'source': 'wos', 'lastUpdated': '2018-11-21 04:01:54.683597+00:00', 'state': 'error-not_actually-failed', 'dispatch': {'timestamp': '2018-11-22 00:46:24.192535+00:00', 'eventId': 'scheduler_placeholder_id', 'pubsub_message_id': '303321092368381'}, 'tasking_id': 'c862c7b3-593e-46f5-82da-2e47b9a0da88', 'parse': {'blob_name': 'v1/grid.9486.3/wos/2016-06/2018-11-22 00:46:24.204666+00:00_1034eb22-d580-4f5a-93d3-078c74ea5500.csv', 'eventId': '

In [287]:
db.collection("tasks").document("1034eb22-d580-4f5a-93d3-078c74ea5500").update({
    u"load.results": 446,
    u"state": u"loaded"
})
db.collection("tasks").document("32aa8cdd-b65d-4d29-be1c-eeea4d2f73ee").update({
    u"state": u"loaded",
    u"load.results": 230
})
db.collection("tasks").document("6603bce6-94df-4fc7-bedb-dd83069431e4").update({
    u"state": u"loaded",
    u"load.results": 740
})
db.collection("tasks").document("6c99f141-423e-4358-99bf-2c1c4a4a23b7").update({
    u"state": u"loaded",
    u"load.results": 257
})

update_time {
  seconds: 1543383906
  nanos: 446140000
}

In [320]:
counts = {
    "wos": 0,
    "scopus": 0,
    "msa": 0
}

task_error_docs = db.collection("tasks").where(u'state', u'==', "loaded").get()
for task in task_error_docs:
    counts[task.to_dict()["source"]] += 1
    print(task.to_dict())
    print(task.id)
    print("")

print(counts["wos"])
print(counts["scopus"])
print(counts["msa"])

0
0
0


In [319]:
db.collection("tasks").document("3b7cc72f-ca7b-450b-a562-a82ad07df984").update({
    u"state": u"open"
})
db.collection("tasks").document("a5ae334a-6ff1-434e-bd54-ee8aa4713182").update({
    u"state": u"open"
})

update_time {
  seconds: 1543554426
  nanos: 662898000
}

In [340]:
qwerty = db.collection("tasks").where(u'state', u'==', "open").get()
count = 0
for task in qwerty:
    count += 1
    
print(count)

0


In [509]:
single = db.collection("tasks").document("22225168-0522-46f9-b2c3-5e7ef838b43d").get()
single_dict = single.to_dict()
pp.pprint(single_dict)

{   'active': True,
    'created': '2018-11-21 04:01:54.390492+00:00',
    'dispatch': {   'eventId': 'scheduler_placeholder_id',
                    'pubsub_message_id': '302870967633969',
                    'timestamp': '2018-11-21 13:56:20.199995+00:00'},
    'externalIdentifier': '60032442',
    'fetch': {   'blob_name': 'v1/grid.9486.3/scopus/2014-06/2018-11-21 '
                              '13:56:20.684890+00:00_22225168-0522-46f9-b2c3-5e7ef838b43d.json',
                 'eventId': '',
                 'timestamp': '2018-11-21 13:56:20.684890+00:00'},
    'gridId': 'grid.9486.3',
    'lastUpdated': '2018-12-12 04:31:19.930756+00:00',
    'load': {   'bigquery_job_id': '605d7482-0130-47c1-af8f-6017ae951a14',
                'eventId': '',
                'results': 329,
                'timestamp': '2018-11-21 13:56:20.684890+00:00'},
    'month': '2014-06',
    'parse': {   'blob_name': 'v2/grid.9486.3/scopus/2014-06/2018-11-21 '
                              '13:56:20.684890

In [360]:
db.collection("tasks").document("1034eb22-d580-4f5a-93d3-078c74ea5599").delete()

seconds: 1544498924
nanos: 386359000

In [358]:
db.collection("tasks").document("1034eb22-d580-4f5a-93d3-078c74ea5599").set({   
    'active': True,
    'created': '2018-11-21 04:01:54.683597+00:00',
    'dispatch': {   'eventId': 'scheduler_placeholder_id',
                    'pubsub_message_id': '303321092368381',
                    'timestamp': '2018-11-22 00:46:24.192535+00:00'},
    'externalIdentifier': 'Universidad Nacional Autonoma de Mexico',
    'fetch': {   'blob_name': 'v1/grid.9486.3/wos/2016-06/2018-11-22 '
                              '00:46:24.204666+00:00_1034eb22-d580-4f5a-93d3-078c74ea5599.json',
                 'eventId': '',
                 'timestamp': '2018-11-22 00:46:24.204666+00:00'},
    'gridId': 'grid.9486.3',
    'lastUpdated': '2018-11-21 04:01:54.683597+00:00',
    'load': {   'bigquery_job_id': 'f0c84549-8cae-450f-b8d0-5bbf879bee47',
                'eventId': '',
                'results': 446,
                'timestamp': '2018-11-22 00:46:24.204666+00:00'},
    'month': '2016-06',
    'parse': {   'blob_name': 'v1/grid.9486.3/wos/2016-06/2018-11-22 '
                              '00:46:24.204666+00:00_1034eb22-d580-4f5a-93d3-078c74ea5500.csv',
                 'eventId': 'placeholder',
                 'timestamp': '2018-11-22 00:46:24.204666+00:00'},
    'previousTask': None,
    'source': 'wos',
    'state': 'fetched',
    'tasking_id': 'c862c7b3-593e-46f5-82da-2e47b9a0da88'
})

update_time {
  seconds: 1544498037
  nanos: 166986000
}

In [ ]:
00503459-f7be-4ff3-8d84-e6276905f999
3b7cc72f-ca7b-450b-a562-a82ad07df999

In [514]:
to_rerun = db.collection("tasks").where(u'gridId', u'==', "grid.434933.a").get()
count = 0
task_ids = {}
for task in to_rerun:
    count += 1
    task_dict = task.to_dict()
    task_ids[task.id] = task_dict['fetch']['blob_name']
    
print(count)
print(task_ids)

475
{'00be6734-6366-419c-abbd-d6a2cc1c1a8c': 'v1/grid.434933.a/wos/2014-12/2018-11-22 01:57:27.082238+00:00_00be6734-6366-419c-abbd-d6a2cc1c1a8c.json', '0109ce5b-900e-4fc8-94dc-4b80351b80d3': 'v1/grid.434933.a/wos/2013-09/2018-12-04 06:08:09.257734+00:00_0109ce5b-900e-4fc8-94dc-4b80351b80d3.json', '0113ddbe-5c85-4802-8a3b-c5b56a3f5c6f': 'v1/grid.434933.a/wos/2001-08/2018-12-04 03:43:00.932293+00:00_0113ddbe-5c85-4802-8a3b-c5b56a3f5c6f.json', '01c40e98-33b7-40a3-a108-a17f0a71a639': 'v1/grid.434933.a/scopus/2015-07/2018-11-21 15:59:21.773745+00:00_01c40e98-33b7-40a3-a108-a17f0a71a639.json', '023c1e0c-9251-480f-bec4-b6297f1eef67': 'v1/grid.434933.a/scopus/2010-12/2018-12-03 09:42:14.146216+00:00_023c1e0c-9251-480f-bec4-b6297f1eef67.json', '03276b5e-680c-4240-9fd6-9482e82c5a4e': 'v1/grid.434933.a/scopus/2002-04/2018-12-03 08:50:13.527398+00:00_03276b5e-680c-4240-9fd6-9482e82c5a4e.json', '0332f343-d6bd-4aa4-97e9-796724620a69': 'v1/grid.434933.a/wos/2016-03/2018-11-22 02:12:28.041230+00:00_0

In [515]:
batch = db.batch()

for task in task_ids:
    batch.update(db.collection(u'tasks').document(task), {
        u'active': True,
        u'state': "fetched"
    })
batch.commit()

[update_time {
   seconds: 1544592282
   nanos: 558504000
 }, update_time {
   seconds: 1544592282
   nanos: 558504000
 }, update_time {
   seconds: 1544592282
   nanos: 558504000
 }, update_time {
   seconds: 1544592282
   nanos: 558504000
 }, update_time {
   seconds: 1544592282
   nanos: 558504000
 }, update_time {
   seconds: 1544592282
   nanos: 558504000
 }, update_time {
   seconds: 1544592282
   nanos: 558504000
 }, update_time {
   seconds: 1544592282
   nanos: 558504000
 }, update_time {
   seconds: 1544592282
   nanos: 558504000
 }, update_time {
   seconds: 1544592282
   nanos: 558504000
 }, update_time {
   seconds: 1544592282
   nanos: 558504000
 }, update_time {
   seconds: 1544592282
   nanos: 558504000
 }, update_time {
   seconds: 1544592282
   nanos: 558504000
 }, update_time {
   seconds: 1544592282
   nanos: 558504000
 }, update_time {
   seconds: 1544592282
   nanos: 558504000
 }, update_time {
   seconds: 1544592282
   nanos: 558504000
 }, update_time {
   second

In [370]:
from google.cloud import pubsub_v1

In [371]:
publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path("coki-214004", "parse-pubsub")

In [516]:
for task in task_ids:
    data = {
        "name": task_ids[task],
        "bucket": "coki-fetched-doi"
    }
    future = publisher.publish(topic_path, data=json.dumps(data).encode('utf-8'))
    message_id = future.result()
    print(message_id)

323401761298545
323400038187181
323401464792116
323401761297496
323400038186137
323401464789458
323401761296448
323400038184095
323401464787393
323401761296400
323400038184063
323401464787360
323401761294358
323400038184030
323401464787307
323401761293309
323400038184009
323401464786227
323401761292262
323400038182973
323401464785189
323401761290618
323400038182862
323401464785144
323401761290596
323400038182790
323401464783068
323401761289509
323400038180749
323401464780186
323401761287467
323400038180723
323401464779388
323401761285552
323400038178163
323401464777998
323401761283392
323400038175130
323401761282371
323400038174107
323401464774676
323401761279343
323400038172910
323401464770459
323401464767452
323400038172876
323401761278237
323401464765938
323400038171834
323401761273674
323401464763796
323400038170748
323401464762742
323401761271991
323400038168729
323401464762664
323401761271231
323400038168701
323401464762617
323401761271184
323400038168672
323401464762575
32340146

In [ ]:
v1/grid.8664.c/wos/2018-05/2018-11-21 07:18:20.708231+00:00_3d8c915b-38e7-47e2-9803-d4ec7bb356d4.json 
v1/grid.8664.c/wos/2018-05/2018-11-21 07:18:20.708231+00:00_3d8c915b-38e7-47e2-9803-d4ec7bb356d4.json
            3d8c915b-38e7-47e2-9803-d4ec7bb356d4
    
v1/grid.8664.c/msa/2015-01/2018-11-21 04:44:34.528091+00:00_f44d93c7-18f3-42f5-91e4-951cf5fc117b.json      
            f44d93c7-18f3-42f5-91e4-951cf5fc117b

In [374]:
for task in task_ids:
    print(task_ids[task])

v1/grid.34980.36/scopus/2001-05/2018-11-28 10:03:23.087351+00:00_004340b3-6d5c-4323-948f-e128a60e46fd.json
v1/grid.34980.36/scopus/2001-02/2018-11-28 10:01:27.573535+00:00_007cc8f3-5a6b-4f0b-83bf-90253739d998.json
v1/grid.34980.36/wos/2000-06/2018-11-28 12:57:16.136223+00:00_00f1d04b-0f8f-452a-80c4-0fd29dcb21d3.json
v1/grid.34980.36/wos/2002-09/2018-11-28 13:24:06.722283+00:00_010b3dca-8d79-4a7e-a394-5e5703e07631.json
v1/grid.34980.36/wos/2000-04/2018-11-28 12:55:14.942942+00:00_019ea9c6-f6fc-4e3d-b076-9dcbf72d201f.json
v1/grid.34980.36/wos/2000-02/2018-11-28 12:53:15.735270+00:00_01af6a29-ac87-4084-b73c-34c2117cec25.json
v1/grid.34980.36/scopus/2017-06/2018-11-21 05:24:20.670533+00:00_024431d0-be98-4fb1-8f13-54ffa05a871d.json
v1/grid.34980.36/msa/2017-01/2018-11-21 04:29:55.704912+00:00_02813da2-42e4-42c6-818c-a4443117caed.json
v1/grid.34980.36/scopus/2018-12/2018-11-21 05:33:20.676704+00:00_03babf5b-2bce-4290-ba80-f11dcdcd6953.json
v1/grid.34980.36/scopus/2004-11/2018-11-28 10:24:31.

In [425]:
to_check_rerun = db.collection("tasks").where(u'gridId', u'==', "grid.34980.36").get()
count = 0
for task in to_check_rerun:
    count += 1
    task_dict = task.to_dict()
    #if "version" in task_dict['parse']:
    #    print(task_dict['parse']["version"], task_dict['parse']["state"])
    #else:
    #    print(task_dict['parse']["state"])
    
    if task_dict['parse']["state"] == "failed":
        print(task.id, task_dict['parse']["version"], task_dict['parse']["state"], task_dict['parse']["blob_name"])
    
    if "version" not in task_dict['parse']:
        print(task.id, task_dict['parse']["state"], task_dict['fetch']["blob_name"])

In [540]:
data = {
    "name": 'v1/grid.7836.a/wos/2015-09/2018-11-27 10:42:05.494588+00:00_29828c62-78bd-466c-9d29-73607f1e3c01.json',
    "bucket": "coki-fetched-doi"
}
future = publisher.publish(topic_path, data=json.dumps(data).encode('utf-8'))
message_id = future.result()
print(message_id)

323416372852068


In [ ]:
2018-11-21 04:29:55.704913+00:00_02813da2-42e4-42c6-818c-a4443117caed.json
2018-11-21 04:29:55.704912+00:00_02813da2-42e4-42c6-818c-a4443117caed.json

In [539]:
db.collection("tasks").document("29828c62-78bd-466c-9d29-73607f1e3c01").update({
    u"fetch": {
        'blob_name': 'v1/grid.7836.a/wos/2015-09/2018-11-27 10:42:05.494588+00:00_29828c62-78bd-466c-9d29-73607f1e3c01.json'
    }
})

update_time {
  seconds: 1544593913
  nanos: 712901000
}

In [431]:
still_hanging = db.collection("tasks").where(u'gridId', u'==', "grid.1032.0").where("state", "==", "parsed").get()
count = 0
task_ids_hanging = {}
for task in still_hanging:
    count += 1
    task_dict = task.to_dict()
    task_ids_hanging[task.id] = task_dict['fetch']['blob_name']
    
print(count)
print(task_ids_hanging)

44
{'9032cb01-6601-4e59-92c5-1336074a29c2': 'v1/grid.1032.0/scopus/2006-12/2018-11-28 07:45:22.966446+00:00_9032cb01-6601-4e59-92c5-1336074a29c2.json', 'a15aebc7-d0c5-471e-afd1-515c8c1409bb': 'v1/grid.1032.0/wos/2015-07/2018-11-27 07:55:06.908199+00:00_a15aebc7-d0c5-471e-afd1-515c8c1409bb.json', 'a15fab81-4c5e-4650-a39c-d25e3cfc3f33': 'v1/grid.1032.0/wos/2014-10/2018-11-21 04:20:24.155656+00:00_a15fab81-4c5e-4650-a39c-d25e3cfc3f33.json', 'a369a4ab-ed60-46f9-b468-d830417df0c9': 'v1/grid.1032.0/scopus/2010-04/2018-11-28 08:05:32.490823+00:00_a369a4ab-ed60-46f9-b468-d830417df0c9.json', 'a3acce7f-de45-4ebc-9438-bd60f133bece': 'v1/grid.1032.0/wos/2016-09/2018-11-21 04:59:21.506298+00:00_a3acce7f-de45-4ebc-9438-bd60f133bece.json', 'a6200099-0de4-4271-813f-421173907b41': 'v1/grid.1032.0/wos/2010-11/2018-11-28 09:24:14.630014+00:00_a6200099-0de4-4271-813f-421173907b41.json', 'a65ab86d-a9fb-43e9-910c-c041a67a9d6a': 'v1/grid.1032.0/msa/2018-01/2018-11-21 04:20:24.456967+00:00_a65ab86d-a9fb-43e9-

In [536]:
to_rerun = db.collection("tasks").where(u'state', u'==', "fetched").get()
count = 0
task_ids = {}
for task in to_rerun:
    count += 1
    task_dict = task.to_dict()
    task_ids[task.id] = {
        "blob_name": task_dict['fetch']['blob_name'],
        "id": task.id,
        "new_blob_name": ""
    }
           
print(count)

1


In [538]:
print(task_ids)

{'29828c62-78bd-466c-9d29-73607f1e3c01': {'blob_name': 'v1/grid.7836.a/wos/2015-09/2018-11-27 10:42:05.494580+00:00_29828c62-78bd-466c-9d29-73607f1e3c01.json', 'id': '29828c62-78bd-466c-9d29-73607f1e3c01', 'new_blob_name': ''}}


In [533]:
for task in task_ids:
    count = 0
    for char in task_ids[task]["blob_name"]:
        if char == '+':
            
            new_char = int(task_ids[task]["blob_name"][count -1])
            new_char += 1
            if new_char == 10:
                new_char = 0
                     
            tmp_list = list(task_ids[task]["blob_name"])
            tmp_list[count -1] = str(new_char)
            task_ids[task]["new_blob_name"] = "".join(tmp_list)
                        
            break
        count += 1
        
    print(task_ids[task]["id"])
    print(task_ids[task]["blob_name"])
    print(task_ids[task]["new_blob_name"])
    print("")
        

05cd7229-9827-43d3-8a0f-aa9510467944
v1/grid.26999.3d/msa/2016-01/2018-11-21 06:39:22.672923+00:00_05cd7229-9827-43d3-8a0f-aa9510467944.json
v1/grid.26999.3d/msa/2016-01/2018-11-21 06:39:22.672924+00:00_05cd7229-9827-43d3-8a0f-aa9510467944.json

0eacd03a-1d50-44d5-87b4-b1109bc7af10
v1/grid.11899.38/msa/2017-01/2018-11-21 07:25:16.549228+00:00_0eacd03a-1d50-44d5-87b4-b1109bc7af10.json
v1/grid.11899.38/msa/2017-01/2018-11-21 07:25:16.549229+00:00_0eacd03a-1d50-44d5-87b4-b1109bc7af10.json

14b894ee-98b4-40c3-a01f-e81ae46471a6
v1/grid.11899.38/msa/2016-01/2018-11-21 07:17:18.380241+00:00_14b894ee-98b4-40c3-a01f-e81ae46471a6.json
v1/grid.11899.38/msa/2016-01/2018-11-21 07:17:18.380242+00:00_14b894ee-98b4-40c3-a01f-e81ae46471a6.json

26e5f778-e6a5-4fb9-bb22-e0f83ffea40e
v1/grid.7836.a/msa/2017-01/2018-11-21 06:17:19.593658+00:00_26e5f778-e6a5-4fb9-bb22-e0f83ffea40e.json
v1/grid.7836.a/msa/2017-01/2018-11-21 06:17:19.593659+00:00_26e5f778-e6a5-4fb9-bb22-e0f83ffea40e.json

29828c62-78bd-466c-9

In [535]:
for task in task_ids:
    print(task_ids[task]["id"])
    print(task_ids[task]["blob_name"])
    print(task_ids[task]["new_blob_name"])
    print("")
    
    db.collection("tasks").document(task_ids[task]["id"]).update({
        u"fetch": {
            'blob_name': task_ids[task]["new_blob_name"]
        }
    })
    
    data = {
        "name": task_ids[task]["new_blob_name"],
        "bucket": "coki-fetched-doi"
    }
    future = publisher.publish(topic_path, data=json.dumps(data).encode('utf-8'))
    message_id = future.result()
    print(message_id)

05cd7229-9827-43d3-8a0f-aa9510467944
v1/grid.26999.3d/msa/2016-01/2018-11-21 06:39:22.672923+00:00_05cd7229-9827-43d3-8a0f-aa9510467944.json
v1/grid.26999.3d/msa/2016-01/2018-11-21 06:39:22.672924+00:00_05cd7229-9827-43d3-8a0f-aa9510467944.json

323411402280731
0eacd03a-1d50-44d5-87b4-b1109bc7af10
v1/grid.11899.38/msa/2017-01/2018-11-21 07:25:16.549228+00:00_0eacd03a-1d50-44d5-87b4-b1109bc7af10.json
v1/grid.11899.38/msa/2017-01/2018-11-21 07:25:16.549229+00:00_0eacd03a-1d50-44d5-87b4-b1109bc7af10.json

323411003673765
14b894ee-98b4-40c3-a01f-e81ae46471a6
v1/grid.11899.38/msa/2016-01/2018-11-21 07:17:18.380241+00:00_14b894ee-98b4-40c3-a01f-e81ae46471a6.json
v1/grid.11899.38/msa/2016-01/2018-11-21 07:17:18.380242+00:00_14b894ee-98b4-40c3-a01f-e81ae46471a6.json

323411402272580
26e5f778-e6a5-4fb9-bb22-e0f83ffea40e
v1/grid.7836.a/msa/2017-01/2018-11-21 06:17:19.593658+00:00_26e5f778-e6a5-4fb9-bb22-e0f83ffea40e.json
v1/grid.7836.a/msa/2017-01/2018-11-21 06:17:19.593659+00:00_26e5f778-e6a5-4